In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, glob
import numpy as np
from tqdm import tqdm
import torch
import smplx
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.transform import Rotation as R
from scipy.ndimage import gaussian_filter1d
import json
import csv
import pdb
import pickle

In [2]:
# need same format as in utils_canonicalize_amass.py??

In [3]:
def get_body_model(type, gender, batch_size,device='cpu'):
    '''
    type: smpl, smplx smplh and others. Refer to smplx tutorial
    gender: male, female, neutral
    batch_size: an positive integar
    '''
    body_model_path = '/home/yuxinyao/body_models/'
    body_model = smplx.create(body_model_path, model_type=type,
                                    gender=gender, ext='npz',
                                    num_pca_comps=12,
                                    create_global_orient=True,
                                    create_body_pose=True,
                                    create_betas=True,
                                    create_left_hand_pose=True,
                                    create_right_hand_pose=True,
                                    create_expression=True,
                                    create_jaw_pose=True,
                                    create_leye_pose=True,
                                    create_reye_pose=True,
                                    create_transl=True,
                                    batch_size=batch_size
                                    )
    if device == 'cuda':
        return body_model.cuda()
    else:
        return body_model
        

In [4]:
with open('/home/yuxinyao/datasets/egobody/smplx_camera_wearer/recording_20210907_S02_S01_01/body_idx_1/results/frame_01551/000.pkl', 'rb') as f:
    data = dict(pickle.load(f))

In [5]:
bm_one_female =get_body_model('smplx','female',1,device='cuda')

In [6]:
transl = data['transl']
pose = data['body_pose']
global_orient = data['global_orient']
betas = data['betas']

In [7]:
global_orient

array([[ 0.9270807 ,  0.37623283, -3.0154164 ]], dtype=float32)

In [8]:
# bodyconfig = {}
# bodyconfig['transl'] = torch.FloatTensor(transl).cuda()
# bodyconfig['global_orient'] = torch.FloatTensor(global_orient).cuda()
# bodyconfig['body_pose'] = torch.FloatTensor(pose).cuda()
# bodyconfig['betas'] = torch.FloatTensor(betas).cuda()
# smplxout = bm_one_female(**bodyconfig)
# joints = smplxout.joints.squeeze().detach().cpu().numpy()
# x_axis = joints[2,:] - joints[1,:]
# x_axis[-1] = 0
# x_axis = x_axis / np.linalg.norm(x_axis)
# z_axis = np.array([0,0,1])
# y_axis = np.cross(z_axis, x_axis)
# y_axis = y_axis/np.linalg.norm(y_axis)
# global_ori_new = np.stack([x_axis, y_axis, z_axis], axis=1)
# transl_new = joints[:1,:] 

# return global_ori_new, transl_new

In [9]:
# get new coordinate

# bodyconfig = {}
# bodyconfig['transl'] = torch.FloatTensor(transl).cuda()
# bodyconfig['global_orient'] = torch.FloatTensor(global_orient).cuda()
# bodyconfig['body_pose'] = torch.FloatTensor(pose).cuda()
# bodyconfig['betas'] = torch.FloatTensor(betas).cuda()

# smplxout = bm_one_female(**bodyconfig)
# joints = smplxout.joints.squeeze().detach().cpu().numpy()
# x_axis = joints[2,:] - joints[1,:]
# x_axis[-1] = 0
# x_axis = x_axis / np.linalg.norm(x_axis)
# z_axis = np.array([0,0,1])
# y_axis = np.cross(z_axis, x_axis)
# y_axis = y_axis/np.linalg.norm(y_axis)
# global_ori_new = np.stack([x_axis, y_axis, z_axis], axis=1)
# transl_new = joints[:1,:] # put the local origin to pelvis


# transf_rotmat = global_ori_new
# transf_transl = transl_new



In [10]:
# #calibrate offset
# # n_batches = 1
# # bodyconfig = {}
# # bodyconfig['body_pose'] = torch.FloatTensor(pose[:,3:]).cuda()
# # bodyconfig['betas'] = torch.FloatTensor(betas).unsqueeze(0).repeat(n_batches,1).cuda()
# # bodyconfig['transl'] = torch.zeros([n_batches,3], dtype=torch.float32).cuda()
# # bodyconfig['global_orient'] = torch.zeros([n_batches,3], dtype=torch.float32).cuda()
# smplx_out = bm_one_female(return_verts=True, **bodyconfig)
# delta_T = smplx_out.joints[:,0,:] # we output all pelvis locations
# delta_T = delta_T.detach().cpu().numpy() #[t, 3]


In [11]:
global_ori = R.from_rotvec(global_orient).as_matrix()


In [12]:
# global_ori.shape

(1, 3, 3)

In [13]:
global_ori_new = np.einsum('ij,tjk->tik', R.from_rotvec([np.pi/2, 0, np.pi]).as_matrix() , global_ori)

In [14]:
# global_ori_new.shape

(1, 3, 3)

In [15]:
temp_bodyconfig = {}
temp_bodyconfig['transl'] = torch.FloatTensor(transl).cuda()
temp_bodyconfig['global_orient'] = torch.FloatTensor(R.from_matrix(global_ori_new).as_rotvec()).cuda()
temp_bodyconfig['body_pose'] = torch.FloatTensor(pose).cuda()
temp_bodyconfig['betas'] = torch.FloatTensor(betas).cuda()

smplxout = bm_one_female(**temp_bodyconfig)
joints = smplxout.joints.squeeze().detach().cpu().numpy()
x_axis = joints[2,:] - joints[1,:]
x_axis[-1] = 0
x_axis = x_axis / np.linalg.norm(x_axis)
z_axis = np.array([0,0,1])
y_axis = np.cross(z_axis, x_axis)
y_axis = y_axis/np.linalg.norm(y_axis)
global_ori_new = np.stack([x_axis, y_axis, z_axis], axis=1)
transl_new = joints[:1,:] # put the local origin to pelvis


transf_rotmat = global_ori_new
transf_transl = transl_new

In [16]:
smplx_out = bm_one_female(return_verts=True, **temp_bodyconfig)
delta_T = smplx_out.joints[:,0,:] # we output all pelvis locations
delta_T = delta_T.detach().cpu().numpy() #[t, 3]

In [17]:
transf_rotmat.shape

(3, 3)

In [18]:
# R.from_matrix(global_ori_new).as_matrix().shape
type(global_ori)
global_ori.shape

(1, 3, 3)

In [19]:
global_ori_new2 =  np.expand_dims(global_ori_new,0)

In [20]:
# b = np.dstack([global_ori_new2]*10)

In [21]:
# b

In [22]:
global_ori_new2

array([[[-0.34426281,  0.93887332,  0.        ],
        [-0.93887329, -0.34426282,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]])

In [23]:
global_ori_new = np.einsum('ij,tjk->tik', transf_rotmat.T, global_ori_new2)

In [24]:
global_ori_new

array([[[0.99999994, 0.        , 0.        ],
        [0.        , 1.        , 0.        ],
        [0.        , 0.        , 1.        ]]])

In [25]:
# global_ori_new = np.einsum('ij,tjk->tik', R.from_rotvec([np.pi, 0, 0]).as_matrix() , global_ori_new)


In [26]:
global_ori_new

array([[[0.99999994, 0.        , 0.        ],
        [0.        , 1.        , 0.        ],
        [0.        , 0.        , 1.        ]]])

In [27]:
# global_ori_new = np.einsum('ij,tjk->tik', R.from_rotvec([0, 0, np.pi]).as_matrix() , global_ori_new)

In [28]:
global_ori_new

array([[[0.99999994, 0.        , 0.        ],
        [0.        , 1.        , 0.        ],
        [0.        , 0.        , 1.        ]]])

In [29]:
global_orient_save = R.from_matrix(global_ori_new).as_rotvec()

In [30]:
global_orient_save#  why not 0 0 0 ?

array([[0., 0., 0.]])

In [31]:
transl2 = transl.repeat(10)

In [32]:
transl_save = np.einsum('ij,tj->ti', transf_rotmat.T, transl+delta_T-transf_transl)-delta_T

In [33]:
transl_save #why not 0,0,0??

array([[-0.87619786,  0.05534796, -0.01364851]])

In [34]:
with open('/home/yuxinyao/body_models/Mosh_related/CMU.json') as f:
        marker_cmu_41 = list(json.load(f)['markersets'][0]['indices'].values())

with open('/home/yuxinyao/body_models/Mosh_related/SSM2.json') as f:
        marker_ssm_67 = list(json.load(f)['markersets'][0]['indices'].values())

In [35]:
data_out = data
data_out['trans'] = transl_save
data_out['body_pose'] = pose
data_out['betas'] = betas
data_out['gender'] = data['gender']
data_out['global_orient'] = global_orient_save



In [36]:
# try save  joints
savetemps = {}
savetemps = data

for key in data.keys():
    if key in ['pose_embedding', 'camera_rotation', 'camera_translation', 'gender']:
        continue
    else:
        savetemps[key] = torch.tensor(data[key]).cuda()

savetemps['trans'] = torch.FloatTensor(transl_save).cuda()
savetemps['body_pose'] = torch.FloatTensor(pose).cuda()
savetemps['betas'] = torch.FloatTensor(betas).cuda()
savetemps['gender'] = data['gender']
savetemps['global_orient'] = torch.FloatTensor(global_orient_save).cuda()


smplxout = bm_one_female(return_verts=True, **savetemps)
joints = smplxout.joints[:,:22,:].detach().squeeze().cpu().numpy()
markers_41 = smplxout.vertices[:,marker_cmu_41,:].detach().squeeze().cpu().numpy()
markers_67 = smplxout.vertices[:,marker_ssm_67,:].detach().squeeze().cpu().numpy()
data_out['joints'] = joints
data_out['marker_cmu_41'] = markers_41
data_out['marker_ssm2_67'] = markers_67

In [37]:
with open('/home/yuxinyao/datasets/egobody/test/canonicalized_xrotzrot5.pkl', 'wb') as f:
    pickle.dump(data_out,f)

In [38]:
print(global_orient_save)
print(data['global_orient'])

[[0. 0. 0.]]
tensor([[0., 0., 0.]], device='cuda:0')


In [39]:
transl_save

array([[-0.87619786,  0.05534796, -0.01364851]])

In [40]:
# global_ori

In [41]:
# smplxout = bm_one_female(**bodyconfig)

In [42]:
# joints = smplxout.joints.squeeze(0)

In [43]:
# jointNumpy = joints.detach().cpu().numpy()

In [44]:
# j = joints[:24]

In [45]:
# jnp = jointNumpy[:24]

In [46]:
# #cal calibrate offset
# bodyconfig = {}
# bodyconfig['body_pose'] = torch.FloatTensor(pose).cuda()
# bodyconfig['betas'] = torch.FloatTensor(betas).cuda()
# bodyconfig['transl'] = torch.zeros([1,3], dtype=torch.float32).cuda()
# bodyconfig['global_orient'] = torch.zeros([1,3], dtype=torch.float32).cuda()
# smplx_out = bm_one_female(return_verts=True, **bodyconfig)
# delta_T = smplx_out.joints[:,0,:] # we output all pelvis locations
# delta_T = delta_T.detach().cpu().numpy() #[t, 3]
# global_ori = R.from_rotvec(pose[:,:3]).as_matrix() # to [t,3,3] rotation mat
# global_ori_new = np.einsum('ij,tjk->tik', transf_rotmat.T, global_ori)
# pose[:,:3] = R.from_matrix(global_ori_new).as_rotvec()


# # return delta_T


In [47]:
# transf_rotmat, transf_transl = get_new_coordinate(bm_one_female, betas[:10], transl[:1,:], pose[:1,:66])

In [48]:
os.getcwd()

'/home/yuxinyao/GAMMA-release/exp_GAMMAPrimitive'

In [49]:
os.chdir('/home/yuxinyao/GAMMA-release/')

In [50]:
import os
import sys
import numpy as np
import open3d as o3d
import torch
import smplx
import cv2
import pickle
import pdb
import re
import glob

# sys.path.append(os.getcwd())
from exp_GAMMAPrimitive.utils.batch_gen_amass import BatchGeneratorAMASSCanonicalized
from exp_GAMMAPrimitive.utils.vislib import *
from exp_GAMMAPrimitive.vis_GAMMAprimitive import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [51]:
import open3d as o3d

In [53]:
from exp_GAMMAPrimitive.utils.vislib import *

In [54]:

verts = smplxout.vertices.detach().cpu().numpy()
joints = smplxout.joints[:,:22].detach().cpu().numpy()

In [55]:
from zmq import DRAFT_API


np.random.seed(0)
vis = o3d.visualization.Visualizer()
vis.create_window(width=960, height=540,visible=True)
# vis.create_window(width=480, height=270,visible=True)
render_opt=vis.get_render_option()
render_opt.mesh_show_back_face=True
render_opt.line_width=10
render_opt.point_size=5
render_opt.background_color = color_hex2rgb('#1c2434')
vis.update_renderer()

### top lighting
box = o3d.geometry.TriangleMesh.create_box(width=200, depth=1,height=200)
box.translate(np.array([-200,-200,6]))
vis.add_geometry(box)
vis.poll_events()
vis.update_renderer()

#### world coordinate
coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.25)
vis.add_geometry(coord)
vis.poll_events()
vis.update_renderer()


# if show_body  71
body = o3d.geometry.TriangleMesh()
vis.add_geometry(body)
vis.poll_events()
vis.update_renderer()

#100
#116 if show_body 
cv2.namedWindow('frame2')
body.vertices = o3d.utility.Vector3dVector(verts)
body.triangles = o3d.utility.Vector3iVector(bm_one_female.faces)
body.vertex_normals = o3d.utility.Vector3dVector([])
body.triangle_normals = o3d.utility.Vector3dVector([])
body.compute_vertex_normals()
vis.update_geometry(body)

body.paint_uniform_color(color_hex2rgb('#c7624f'))


ctr = vis.get_view_control()
ctr.set_constant_z_far(10)
cam_param = ctr.convert_to_pinhole_camera_parameters()
### get cam T
# body_t = np.mean(data[it],axis=0) # let cam follow the body
body_t = np.array([0,0,0])
cam_t = body_t + 2.0*np.ones(3)
### get cam R
cam_z =  body_t - cam_t
cam_z = cam_z / np.linalg.norm(cam_z)
cam_x = np.array([cam_z[1], -cam_z[0], 0.0])
cam_x = cam_x / np.linalg.norm(cam_x)
cam_y = np.array([cam_z[0], cam_z[1], -(cam_z[0]**2 + cam_z[1]**2)/cam_z[2] ])
cam_y = cam_y / np.linalg.norm(cam_y)
cam_r = np.stack([cam_x, -cam_y, cam_z], axis=1)
### update render cam
transf = np.eye(4)
transf[:3,:3]=cam_r
transf[:3,-1] = cam_t
cam_param = update_render_cam(cam_param, transf)
ctr.convert_from_pinhole_camera_parameters(cam_param)
vis.poll_events()
vis.update_renderer()

## capture RGB appearance
rgb = np.asarray(vis.capture_screen_float_buffer(do_render=True))
cv2.imshow("frame2", np.uint8(255*rgb[:,:,[2,1,0]]))
outfile_path = '/home/yuxinyao/datasets/egobody/test'
renderimgname = os.path.join(outfile_path, 'img1.png')
cv2.imwrite(renderimgname, np.uint8(255*rgb[:,:,[2,1,0]]))
cv2.waitKey(5)




RuntimeError: Unable to cast Python instance to C++ type (compile in debug mode for details)

: 

In [ ]:
# get vertices
